In [2]:
from __future__ import print_function
from bs4 import BeautifulSoup
import requests

In [ ]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36"
}
 
# from 2018 season
url = 'https://www.footballdb.com/statistics/nfl/player-stats/kicking/2018/regular-season?sort=kickpts&limit=100'
req = requests.get(url, headers=headers)
soup = BeautifulSoup(req.text, 'html.parser')

print(soup)

In [4]:
# content
content = soup.find(id='leftcol')
# headers
headers = content.find_all('th')
# getting column names
cols = [head.text.strip() for head in headers if head.text.strip()]
print(cols)

['Player', 'Team', 'Gms', 'PAT', 'FG', '0-19', '20-29', '30-39', '40-49', '50+', 'Lg', 'Pts']


In [86]:
import pandas as pd
# stats
table = content.find_all('tr')
# converting to dataframe
res = []
for tr in table:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        res.append(row)
# data
data = pd.DataFrame(res, columns=cols)
data.head()

,Player,Team,Gms,PAT,FG,0-19,20-29,30-39,40-49,50+,Lg,Pts
0,Ka'imi FairbairnK. Fairbairn,HOU,16,39/41,37/42,1/1,9/9,11/11,12/15,4/6,54,150
1,Justin TuckerJ. Tucker,BAL,16,36/37,35/39,0/0,12/12,9/9,9/11,5/7,56,141
2,Harrison ButkerH. Butker,KC,16,65/69,24/27,0/0,9/9,7/7,6/7,2/4,54,137
3,Wil LutzW. Lutz,NO,16,52/53,28/30,1/1,5/5,9/9,11/12,2/3,54,136
4,Stephen GostkowskiS. Gostkowski,NE,16,49/50,27/32,0/0,11/11,10/10,4/6,2/5,52,130


In [82]:
# checking element type
data["Gms"] = pd.to_numeric(data["Gms"])
data["Lg"] = pd.to_numeric(data["Lg"])
data["Pts"] = pd.to_numeric(data["Pts"])

In [138]:
def split(csv,col):
  # splitting
  split = csv[col].str.split("/", expand=True)
  # adding
  csv[col] = pd.to_numeric(split[0])/pd.to_numeric(split[1])
  return

In [ ]:
print("shape before: ",data.shape)
data = data.fillna(data.mean())
print("shape after: ",data.shape)
data.head()

In [160]:
import pandas as pd
# stats
table = content.find_all('tr')
# converting to dataframe
res = []
for tr in table:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        res.append(row)

# data
data = pd.DataFrame(res, columns=cols)
data.head()

# renaming 
pivot_data = pd.DataFrame(res, columns=cols)
pivot_data.rename(columns={"0-19": "19", "20-29": "29", "30-39":"39","40-49":"49","50+":"59"}, inplace=True)

# numerical
csv = pivot_data
split(csv,"PAT")
split(csv,"FG")
split(csv,"19")
split(csv,"29")
split(csv,"39")
split(csv,"49")
split(csv,"59")
# filling

# filtering
pivot_data = pivot_data[['19','29','39','49','59']]

pivot_data.fillna(pivot_data.mean(), inplace=True)

pivot_data.head()

distance = []
kicks = []
# pivoting
for column in pivot_data:
  for row in range(len(pivot_data)):
    # adding data
    distance.append(int(column))
    kicks.append(pivot_data[column][row])

final = pd.DataFrame()
final['Distance'] = distance
final['Kicks'] = kicks

final.head()

<ipython-input-160-1a807a975459>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pivot_data.fillna(pivot_data.mean(), inplace=True)
<ipython-input-160-1a807a975459>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pivot_data.fillna(pivot_data.mean(), inplace=True)
<ipython-input-160-1a807a975459>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydat

,Distance,Kicks
0,19,1.0
1,19,1.0
2,19,1.0
3,19,1.0
4,19,1.0


In [184]:
from sklearn import linear_model
import numpy as np

# features and labels
X = np.array(final['Distance']).reshape((-1, 1))
y = final['Kicks']

# model
model = linear_model.LinearRegression()
model.fit(X, y)

# input
predict = 50

# predicting
predict =  np.array([predict]).reshape((-1, 1))
prediction= model.predict(predict)

# printing
print("kick success likeliness: ",prediction[0])

kick success likeliness:  0.7399703923332053


In [181]:
import statsmodels.api as sm
import numpy as np

# Create some example data
X = np.array(final['Distance']).reshape((-1, 1))
y = final['Kicks']

# Fit a logistic regression model
model = sm.GLM(y, sm.add_constant(X), family=sm.families.Binomial())
result = model.fit()

# Print the model summary
print(result.summary())

# 

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  Kicks   No. Observations:                  205
Model:                            GLM   Df Residuals:                      203
Model Family:                Binomial   Df Model:                            1
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -53.421
Date:                Fri, 17 Mar 2023   Deviance:                       51.459
Time:                        16:00:51   Pearson chi2:                     75.2
No. Iterations:                     6   Pseudo R-squ. (CS):             0.1715
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          6.7863      1.163      5.837      0.0

In [185]:
import statsmodels.api as sm

X = np.array(final['Distance']).reshape((-1, 1))
y = final['Kicks']
X = sm.add_constant(X)  # add an intercept term
model = sm.OLS(y, X).fit()

# Get a summary of the model
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kicks   R-squared:                       0.347
Model:                            OLS   Adj. R-squared:                  0.344
Method:                 Least Squares   F-statistic:                     108.1
Date:                Fri, 17 Mar 2023   Prob (F-statistic):           1.45e-20
Time:                        16:02:29   Log-Likelihood:                 42.577
No. Observations:                 205   AIC:                            -81.15
Df Residuals:                     203   BIC:                            -74.51
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.2471      0.040     30.811      0.0